In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/MyDrive/PAKDD/ShotDetection

Mounted at /content/gdrive
/content/gdrive/MyDrive/PAKDD/ShotDetection


In [7]:
import cv2
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import namedtuple


In [13]:
def format_crt_pts(df_crt_rel, relax=0.15):
    # format court points
    crt_dict = df_crt_rel.to_dict('records')
    crt_pts = np.zeros((4,2))
    # fill up full court points
    crt_pts[0][0], crt_pts[0][1], crt_pts[1][0], crt_pts[1][1], crt_pts[2][0], crt_pts[2][1], crt_pts[3][0], crt_pts[3][1] = \
        (1-relax)*crt_dict[0]['upleft_x'], (1-relax)*crt_dict[0]['upleft_y'], \
        (1+relax)*crt_dict[0]['upright_x'], (1-relax)*crt_dict[0]['upright_y'], \
        (1+relax)*crt_dict[0]['downright_x'], (1+relax)*crt_dict[0]['downright_y'], \
        (1-relax)*crt_dict[0]['downleft_x'], (1+relax)*crt_dict[0]['downleft_y']
    # fill up court points for only the near half
    crt_pts_half_near = crt_pts.copy()
    crt_pts_half_near[0][0], crt_pts_half_near[0][1] = 0.5*(crt_pts[0][0] + crt_pts[3][0]), 0.5*(crt_pts[0][1] + crt_pts[3][1])
    crt_pts_half_near[1][0], crt_pts_half_near[1][1] = 0.5*(crt_pts[1][0] + crt_pts[2][0]), 0.5*(crt_pts[1][1] + crt_pts[2][1])
    # fill up court points for only the far half
    crt_pts_half_far = crt_pts.copy()
    crt_pts_half_far[3][0], crt_pts_half_far[3][1] = crt_pts_half_near[0][0], crt_pts_half_near[0][1]
    crt_pts_half_far[2][0], crt_pts_half_far[2][1] = crt_pts_half_near[1][0], crt_pts_half_near[1][1]

    crt_pts = np.float32(crt_pts.reshape((-1,1,2)))
    crt_pts_half_near = np.float32(crt_pts_half_near.reshape((-1,1,2)))
    crt_pts_half_far = np.float32(crt_pts_half_far.reshape((-1,1,2)))
    
    return crt_pts, crt_pts_half_near, crt_pts_half_far

def create_dummy_rowdict(fr):
    row = {}
    row['Frame'] = fr
    row['x1'], row['y1'], row['x2'], row['y2'], row['bbox_conf'] = 0, 0, 0, 0, 0
    for p in range(17):
        row[str(p) + '_x'], row[str(p) + '_y'] = 0, 0
        
    return row

def get_filtered_pose(df_raw, df_crt, relax=0.15):
    df_crt_rel = df_crt[['upleft_x', 'upleft_y', 'upright_x', 'upright_y', 'downright_x', 'downright_y', 'downleft_x', 'downleft_y']]
    df_crt_rel.at[0, 'upleft_x']= df_crt_rel['upleft_x']/1280
    df_crt_rel.at[0, 'upright_x']= df_crt_rel['upright_x']/1280
    df_crt_rel.at[0, 'downright_x']= df_crt_rel['downright_x']/1280
    df_crt_rel.at[0, 'downleft_x']= df_crt_rel['downleft_x']/1280
    df_crt_rel.at[0, 'upleft_y']= df_crt_rel['upleft_y']/720
    df_crt_rel.at[0, 'upright_y']= df_crt_rel['upright_y']/720
    df_crt_rel.at[0, 'downright_y']= df_crt_rel['downright_y']/720
    df_crt_rel.at[0, 'downleft_y']= df_crt_rel['downleft_y']/720

    # format court points
    crt_pts, crt_pts_half_near, crt_pts_half_far = format_crt_pts(df_crt_rel, relax=relax)
    print(df_crt_rel)
    total_frame = df_raw['Frame'].to_numpy()[-1] + 1
    outrows_list = []
    for fr in range(total_frame):
        df_raw_tmp = df_raw[df_raw['Frame'] == fr].sort_values(by=['bbox_conf'], ascending=False) # sort by confidence
        listdict_raw = df_raw_tmp.to_dict('records')

        # select rows that lie within court
        selrows_list = []
        for rowi in listdict_raw:
            left_foot_coord = (rowi['16_x'], rowi['16_y'])
            # check if point is in crt. 1- in, 0 -on line, -1 out
            in_crt = cv2.pointPolygonTest(crt_pts, left_foot_coord, False)
            if in_crt != -1:
                selrows_list.append(rowi)
        # print(len(selrows_list))
        # identify near and far row
        # CASE 1
        if len(selrows_list) >= 2:
            # check which row is which id based on distance from camera
            if selrows_list[0]['16_y'] > selrows_list[1]['16_y']:
                near_row, far_row = selrows_list[0], selrows_list[1]
            else:
                near_row, far_row = selrows_list[1], selrows_list[0]
        # CASE 2
        elif len(selrows_list) == 1:
            left_foot_coord = (selrows_list[0]['16_x'], selrows_list[0]['16_y'])
            # check if point is in crt. 1- in, 0 -on line, -1 out
            in_near_crt = cv2.pointPolygonTest(crt_pts_half_near, left_foot_coord, False)
            if in_near_crt != -1:
                near_row = selrows_list[0]

                # reuse far row from previous frame, or set to zero if not found
                try:
                    far_row['Frame'] = fr
                except NameError:
                    far_row = create_dummy_rowdict(fr)
            else:
                far_row = selrows_list[0]

                # reuse near row from previous frame, or set to zero if not found
                try:
                    near_row['Frame'] = fr
                except NameError:
                    near_row = create_dummy_rowdict(fr)
        # CASE 3
        else:
            # reuse near and far rows from previous
            try:
                near_row['Frame'] = fr
                far_row['Frame'] = fr
            except NameError:
                far_row = create_dummy_rowdict(fr)
                near_row = create_dummy_rowdict(fr)

        near_row['id'] = 1
        far_row['id'] = 2

        outrows_list.append(near_row.copy())
        outrows_list.append(far_row.copy())

    df_out = pd.DataFrame(outrows_list)
    return df_out


def save_filtered_pose(basename, relax=0.15):
    df_crt = pd.read_csv(basename+'_court.csv')

    # crt_orig_path = os.path.join(crt_orig_rootpath, matchdir, 'rally_video', basename + '.mp4')
    rawcsv = basename+'_pose_bbox.csv'
    outcsv = basename + '_filtered2pose.csv'
    print(outcsv, ' saved')

    df_raw = pd.read_csv(rawcsv)

    df_out = get_filtered_pose(df_raw, df_crt, relax=relax)
    # print(df)
    df_out.to_csv(outcsv, index=False)

In [14]:

# data_dir  = os.path.join(cur_dir, 'dataset', 'aicup_dataset', m)

# hmtx_path = os.path.join(cur_dir, 'dataset', 'homography_matrix.csv')
basename = '1_00_01'
save_filtered_pose(basename, relax=0.15)

1_00_01_filtered2pose.csv  saved
   upleft_x  upleft_y  upright_x  upright_y  downright_x  downright_y  \
0     0.315  0.543889   0.679844   0.546944     0.782656     0.947222   

   downleft_x  downleft_y  
0     0.21625    0.943611  


In [20]:
def combine_domain_into_x(df_crt, df_pose, df_shuttle, id1=1, id2=2):
    df_court_rel = df_crt[['upleft_x', 'upleft_y', 'upright_x', 'upright_y', 'downright_x', 'downright_y', 'downleft_x', 'downleft_y']]
    print(df_court_rel.to_numpy())
    court_array = df_court_rel.to_numpy()[0]
    w = 1280
    h = 720

    total_frame = df_pose['Frame'].to_numpy()[-1] + 1
    outrows_list = []
    for fr in range(total_frame):
        row_dict = {}
        row_dict['Frame'] = fr

        # fill court coordinates
        row_dict['tl_x'], row_dict['tl_y'], row_dict['tr_x'], row_dict['tr_y'], row_dict['br_x'], row_dict['br_y'], row_dict['bl_x'], row_dict['bl_y'] = \
            court_array[0]/w, court_array[1]/h, court_array[2]/w, court_array[3]/h, court_array[4]/w, court_array[5]/h, court_array[6]/w, court_array[7]/h,   

        df_shuttle_tmp = df_shuttle[df_shuttle['Frame'] == fr]
        df_pose_tmp = df_pose[df_pose['Frame'] == fr]

        # get shuttle coordinates
        if not df_shuttle_tmp.empty:
            shuttle_coords = df_shuttle_tmp[['X', 'Y']].to_numpy()[0]
            # rescale shuttle coord
            shuttle_coords = [shuttle_coords[0]/w, shuttle_coords[1]/h]
        else:
            shuttle_coords = [0, 0]
        row_dict['ball_x'], row_dict['ball_y'] = shuttle_coords[0], shuttle_coords[1]
        
        # get bbox and visibility
        bbox_near = df_pose_tmp[df_pose_tmp['id'] == id1][['x1', 'y1', 'x2', 'y2']].to_numpy()[0]
        bbox_far = df_pose_tmp[df_pose_tmp['id'] == id2][['x1', 'y1', 'x2', 'y2']].to_numpy()[0]
        # check visibility, visible if bbox is not all zeroes
        vis_near = 1 if np.any(bbox_near) else 0
        vis_far = 1 if np.any(bbox_far) else 0
        row_dict['vis_near'], row_dict['vis_far'] = vis_near, vis_far
        row_dict['near_x1'], row_dict['near_y1'], row_dict['near_x2'], row_dict['near_y2'] = bbox_near
        row_dict['far_x1'], row_dict['far_y1'], row_dict['far_x2'], row_dict['far_y2'] = bbox_far

        # get pose coordinates
        coords_pose_near = df_pose_tmp[df_pose_tmp['id'] == id1].drop(columns=['Frame', 'id', 'x1', 'y1', 'x2', 'y2', 'bbox_conf']).to_numpy()[0]
        coords_pose_far = df_pose_tmp[df_pose_tmp['id'] == id2].drop(columns=['Frame', 'id', 'x1', 'y1', 'x2', 'y2', 'bbox_conf']).to_numpy()[0]
        for player in range(2):
            rootname = 'near_pose' if player==0 else 'far_pose'
            coords_pose = coords_pose_near if player==0 else coords_pose_far
            for p in range(17):
                row_dict[rootname + str(p) + '_x'], row_dict[rootname + str(p) + '_y'] =  coords_pose[2*p], coords_pose[2*p+1]

        outrows_list.append(row_dict.copy())
    df_out = pd.DataFrame(outrows_list)  

    return df_out

In [21]:
annot_court_file = basename + '_court.csv'
annot_pose_file = basename + '_filtered2pose.csv'
annot_shuttle_file = basename + '_ball.csv'

df_pose = pd.read_csv(annot_pose_file)
df_shuttle = pd.read_csv(annot_shuttle_file)


out_file_x = basename + '_x.csv'

df_crt = pd.read_csv(annot_court_file)

df_out_x = combine_domain_into_x(df_crt, df_pose, df_shuttle)
df_out_x.to_csv(out_file_x, index=False)

[[ 403.2  391.6  870.2  393.8 1001.8  682.   276.8  679.4]]
